In [33]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [34]:
import pandas as pd
import numpy as np
from nltk import tokenize
from datetime import datetime
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from numba import jit, prange

In [35]:
nvidia_df = pd.read_csv("../00_data/Cleaned_Data_22MAR.gz", compression="gzip")

In [36]:
nvidia_df

,Unnamed: 0,Index,videoID,datePub,searchedDate,VideoTitle,channelTitle,viewCount,likeCount,dislikeCount,captionString
0,0,0,y4fb_R5Ogw0,2020-10-01 05:16:02+00:00,2020-10-02 01:45:21.553469+00:00,"2x NVIDIA RTX 3090 SLI Benchmarks: 500FPS, 700...",Gamers Nexus,367472,15701,260,so this setup that you're seeing is what we ha...
1,1,1,ALEXVtnNEwA,2020-09-01 17:15:57+00:00,2020-10-02 01:45:22.351801+00:00,NVIDIA GeForce Special Event Livestream,IGN,543115,16860,411,b welcome to my kitchen i hope all of you are ...
2,2,2,E98hC9e__Xs,2020-09-01 16:42:07+00:00,2020-10-02 01:45:23.252012+00:00,NVIDIA GeForce RTX 30 Series | Official Launch...,NVIDIA GeForce,1484075,54741,1135,Welcome to my kitchen. I hope all of you are...
3,3,3,nMns8t4OhI0,2020-09-30 23:00:06+00:00,2020-10-02 01:45:23.922084+00:00,NVIDIA Responds to RTX 3080 Crashes...,JayzTwoCents,561238,27766,885,NaN
4,4,4,AG_ZHi3tuyk,2020-09-16 12:59:59+00:00,2020-10-02 01:45:24.592244+00:00,"Nvidia, you PROMISED! - RTX 3080 Review",Linus Tech Tips,3391565,141852,3961,- The RTX 3080 is the most powerful GPU we've ...
...,...,...,...,...,...,...,...,...,...,...,...
20788,20788,20,AG_ZHi3tuyk,2020-09-16 12:59:59+00:00,2021-03-19 18:03:20.670031+00:00,"Nvidia, you PROMISED! - RTX 3080 Review",Linus Tech Tips,3741789,148430,4216,- The RTX 3080 is the most powerful GPU we've ...
20789,20789,21,3dJhDqLjPN0,2021-02-24 21:59:34+00:00,2021-03-19 18:03:21.320922+00:00,NVIDIA (NVDA) Stock Analysis - Q4 Earnings + I...,The Investor Channel,7773,269,1,what is going on investors hopefully guys had ...
20790,20790,22,H1DApIvOCMw,2021-02-26 17:35:37+00:00,2021-03-19 18:03:21.788746+00:00,My SALTY Review of the NVIDIA RTX 3060...,JayzTwoCents,1013948,51773,702,with no parts markup and only a 75 build fee r...
20791,20791,23,Y5J2bgXiK9g,2021-03-09 17:00:10+00:00,2021-03-19 18:03:22.387116+00:00,Should Nvidia Stop Selling 3060s?,LMG Clips,53940,2238,33,uh the you know the 30 80 eth mining wait isn'...


In [ ]:
nvidia_df["Date"].value_counts().sort_index()

In [38]:
nvidia_df.columns

Index(['Unnamed: 0', 'Index', 'videoID', 'datePub', 'searchedDate',
       'VideoTitle', 'channelTitle', 'viewCount', 'likeCount', 'dislikeCount',
       'captionString'],
      dtype='object')

In [39]:
# initialize correct types of data for each column
nvidia_df["VideoTitle"] = nvidia_df["VideoTitle"].astype("str")
nvidia_df["datePub"] = pd.to_datetime(
    nvidia_df["datePub"], format="%Y-%m-%d %H:%M:%S", utc=True
)
nvidia_df["searchedDate"] = pd.to_datetime(
    nvidia_df["searchedDate"], format="%Y-%m-%d %H:%M:%S", utc=True
)
nvidia_df["channelTitle"] = nvidia_df["channelTitle"].astype("str")
nvidia_df["viewCount"] = nvidia_df["viewCount"].astype("int")
nvidia_df["likeCount"] = nvidia_df["likeCount"].astype("int")
nvidia_df["dislikeCount"] = nvidia_df["dislikeCount"].astype("int")
nvidia_df["captionString"] = nvidia_df["captionString"].astype("str")

In [40]:
type(nvidia_df["VideoTitle"][0])

str

In [41]:
# X Variables = Aggregate the following values by search date/time
#     videoTitleSentimentScore_pos = vader pos label for video title
#     captionSentimentScore_pos = vader pos label for caption string
#     XXX - viewcounts_weighted = views count / number of day since published
#     XXX - like_count_weighted = like count / number of day since published
#     XXX - dislike_count_weighted = dislike count / number of day since published
#     XXX - like_dislike_ratio = like / (like + dislike)
#     XXX - weighted_like_dislike_ratio = weighted_like / (weighted_like + weighted_dislike)
#     XXX - like_dislike_ratio_by_viewcounts_weighted = like dislike ration (video potential) * viewcount_weight

In [42]:
# load vader sentiment for comparing the uncleaned strings to the modified ones
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [43]:
# print(f'Title to score = {nvidia_df["captionString"][5]}')
score = sid.polarity_scores(str(nvidia_df["captionString"][5]))
print(score)

{'neg': 0.037, 'neu': 0.846, 'pos': 0.117, 'compound': 0.9994}


In [45]:
nvidia_df["videoTitleSentimentScore_pos"] = [
    sid.polarity_scores(x)["pos"] for x in nvidia_df["VideoTitle"]
]

nvidia_df["videoTitleSentimentScore_neg"] = [
    sid.polarity_scores(x)["neg"] for x in nvidia_df["VideoTitle"]
]

nvidia_df["captionSentimentScore_pos"] = [
    sid.polarity_scores(x)["pos"] for x in nvidia_df["captionString"]
]

nvidia_df["captionSentimentScore_neg"] = [
    sid.polarity_scores(x)["neg"] for x in nvidia_df["captionString"]
]

nvidia_df["captionSentimentScore_com"] = [
    sid.polarity_scores(x)["compound"] for x in nvidia_df["captionString"]
]

In [48]:
nvidia_df["videoTitleSentimentScore_com"] = [
    sid.polarity_scores(x)["compound"] for x in nvidia_df["VideoTitle"]
]

In [62]:
nvidia_df.tail(10)
nvidia_df.to_csv("../00_data/Sentiment_Only_Data_22MAR.gz", compression="gzip")

In [64]:
nvidia_df.head()

,Unnamed: 0,Index,videoID,datePub,searchedDate,VideoTitle,channelTitle,viewCount,likeCount,dislikeCount,...,captionSentimentScore_com,videoTitleSentimentScore_com,num_days_since_pub,viewcounts_weighted,like_count_weighted,dislike_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted,Date
0,0,0,y4fb_R5Ogw0,2020-10-01 05:16:02+00:00,2020-10-02 01:45:21.553469+00:00,"2x NVIDIA RTX 3090 SLI Benchmarks: 500FPS, 700...",Gamers Nexus,367472,15701,260,...,0.9997,0.2023,1.0,367472.000000,15701.000000,260.000000,0.983710,0.983710,361485.989098,2020-10-02
1,1,1,ALEXVtnNEwA,2020-09-01 17:15:57+00:00,2020-10-02 01:45:22.351801+00:00,NVIDIA GeForce Special Event Livestream,IGN,543115,16860,411,...,1.0000,0.4019,31.0,17519.838710,543.870968,13.258065,0.976203,0.976203,17102.917066,2020-10-02
2,2,2,E98hC9e__Xs,2020-09-01 16:42:07+00:00,2020-10-02 01:45:23.252012+00:00,NVIDIA GeForce RTX 30 Series | Official Launch...,NVIDIA GeForce,1484075,54741,1135,...,1.0000,0.0000,31.0,47873.387097,1765.838710,36.612903,0.979687,0.979687,46900.942857,2020-10-02
3,3,3,nMns8t4OhI0,2020-09-30 23:00:06+00:00,2020-10-02 01:45:23.922084+00:00,NVIDIA Responds to RTX 3080 Crashes...,JayzTwoCents,561238,27766,885,...,0.0000,0.0000,2.0,280619.000000,13883.000000,442.500000,0.969111,0.969111,271950.966947,2020-10-02
4,4,4,AG_ZHi3tuyk,2020-09-16 12:59:59+00:00,2020-10-02 01:45:24.592244+00:00,"Nvidia, you PROMISED! - RTX 3080 Review",Linus Tech Tips,3391565,141852,3961,...,0.9997,0.5461,16.0,211972.812500,8865.750000,247.562500,0.972835,0.972835,206214.585797,2020-10-02


In [63]:
nvidia_df["num_days_since_pub"] = np.ceil(
    (nvidia_df["searchedDate"] - nvidia_df["datePub"]) / np.timedelta64(1, "D")
)
nvidia_df["viewcounts_weighted"] = (
    nvidia_df["viewCount"] / nvidia_df["num_days_since_pub"]
)
nvidia_df["like_count_weighted"] = (
    nvidia_df["likeCount"] / nvidia_df["num_days_since_pub"]
)
nvidia_df["dislike_count_weighted"] = (
    nvidia_df["dislikeCount"] / nvidia_df["num_days_since_pub"]
)
nvidia_df["like_dislike_ratio"] = nvidia_df["likeCount"] / (
    nvidia_df["likeCount"] + nvidia_df["dislikeCount"]
)
nvidia_df["weighted_like_dislike_ratio"] = nvidia_df["like_count_weighted"] / (
    nvidia_df["like_count_weighted"] + nvidia_df["dislike_count_weighted"]
)
nvidia_df["like_dislike_ratio_by_viewcounts_weighted"] = (
    nvidia_df["like_dislike_ratio"] * nvidia_df["viewcounts_weighted"]
)
nvidia_df["Date"] = pd.to_datetime(
    nvidia_df["searchedDate"], format="%Y-%m-%d", utc=True
).dt.date

In [59]:
nvidia_df.columns

Index(['Unnamed: 0', 'Index', 'videoID', 'datePub', 'searchedDate',
       'VideoTitle', 'channelTitle', 'viewCount', 'likeCount', 'dislikeCount',
       'captionString', 'videoTitleSentimentScore_pos',
       'videoTitleSentimentScore_neg', 'captionSentimentScore_pos',
       'captionSentimentScore_neg', 'captionSentimentScore_com',
       'videoTitleSentimentScore_com', 'num_days_since_pub',
       'viewcounts_weighted', 'like_count_weighted', 'dislike_count_weighted',
       'like_dislike_ratio', 'weighted_like_dislike_ratio',
       'like_dislike_ratio_by_viewcounts_weighted', 'Date'],
      dtype='object')

In [56]:
X_features = nvidia_df.filter(
    [
        "videoTitleSentimentScore_pos",
        "videoTitleSentimentScore_neg",
        "captionSentimentScore_pos",
        "captionSentimentScore_neg",
        "num_days_since_pub",
        "viewcounts_weighted",
        "like_count_weighted",
        "like_dislike_ratio",
        "weighted_like_dislike_ratio",
        "like_dislike_ratio_by_viewcounts_weighted",
        "Date",
    ],
    axis=1,
)

In [57]:
X_feautures_pre_merge = X_features.groupby(["Date"]).mean().reset_index()

In [58]:
X_feautures_pre_merge.tail()

,Date,videoTitleSentimentScore_pos,videoTitleSentimentScore_neg,captionSentimentScore_pos,captionSentimentScore_neg,num_days_since_pub,viewcounts_weighted,like_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted
113,2021-03-15,0.037338,0.033960,0.087764,0.030724,109.768889,15241.801392,1049.344298,0.955442,0.955442,14861.456219
114,2021-03-16,0.037245,0.064280,0.105100,0.034100,108.970000,14870.723784,1032.050839,0.954523,0.954523,14469.862263
115,2021-03-17,0.038080,0.066406,0.102891,0.033063,111.137143,14748.626569,939.470945,0.956001,0.956001,14378.400682
116,2021-03-18,0.051550,0.061970,0.099855,0.031660,107.740000,15558.251680,1049.395704,0.956913,0.956913,15162.712790
117,2021-03-19,0.042326,0.062851,0.109966,0.036794,110.005714,15417.660309,1121.427479,0.957292,0.957292,15049.514412


In [61]:
# This is a function to generate response variable dataframe
# Decision rule:
# If the change in "High" in 5 days is greater than 3%, we give it label "buy"
# If the change in "High" in 5 days is smaller than -3%, we give it label "sell"
# Otherwise, we give it label "hold
def create_response(start_date, end_date, stock_name):
    import pandas as pd
    import numpy as np
    import yfinance as yf

    # get stock info
    data = (
        yf.download(stock_name, start=start_date, end=end_date)
        .asfreq("D")
        .fillna(method="ffill")
    )
    # create percentage dict
    percent_dict = {}
    for i in range(len(data) - 5):
        cur_high = data.iloc[i, :]["High"]
        future_high = data.iloc[i + 5, :]["High"]
        percent_dict[data.index[i]] = (future_high - cur_high) / cur_high
    response_df = pd.DataFrame.from_dict(
        percent_dict, orient="index", columns=["percent_change"]
    )
    # Decsion rule
    bins = [-(np.inf), -0.03, 0.03, np.inf]
    names = ["sell", "hold", "buy"]
    response_df["action"] = pd.cut(response_df["percent_change"], bins, labels=names)
    response_df = response_df.reset_index()
    return response_df

In [21]:
responce_df = create_response("2020-10-02", "2021-03-28", "NVDA")

[*********************100%***********************]  1 of 1 completed


In [23]:
responce_df.tail()

,index,percent_change,action
162,2021-03-13,0.022908,hold
163,2021-03-14,0.002541,hold
164,2021-03-15,-0.021599,hold
165,2021-03-16,-0.043737,sell
166,2021-03-17,-0.006551,hold


In [24]:
X_feautures_pre_merge["Date"] = pd.to_datetime(
    X_feautures_pre_merge["Date"], format="%Y-%m-%d", utc=True
).dt.date

responce_df["index"] = pd.to_datetime(
    responce_df["index"], format="%Y-%m-%d", utc=True
).dt.date

In [25]:
type(X_feautures_pre_merge["Date"][0])
X_feautures_pre_merge["Date"].tail(50)

68     2021-01-16
69     2021-01-17
70     2021-01-18
71     2021-01-19
72     2021-01-20
73     2021-01-21
74     2021-01-23
75     2021-01-28
76     2021-01-29
77     2021-01-30
78     2021-02-01
79     2021-02-02
80     2021-02-04
81     2021-02-06
82     2021-02-07
83     2021-02-08
84     2021-02-10
85     2021-02-11
86     2021-02-14
87     2021-02-15
88     2021-02-16
89     2021-02-18
90     2021-02-19
91     2021-02-20
92     2021-02-22
93     2021-02-23
94     2021-02-24
95     2021-02-25
96     2021-02-26
97     2021-02-27
98     2021-02-28
99     2021-03-01
100    2021-03-02
101    2021-03-03
102    2021-03-04
103    2021-03-05
104    2021-03-06
105    2021-03-07
106    2021-03-08
107    2021-03-09
108    2021-03-10
109    2021-03-11
110    2021-03-12
111    2021-03-13
112    2021-03-14
113    2021-03-15
114    2021-03-16
115    2021-03-17
116    2021-03-18
117    2021-03-19
Name: Date, dtype: object

In [26]:
type(responce_df["index"][0])
responce_df["index"][0]

datetime.date(2020, 10, 2)

In [27]:
X_feautures_final = X_feautures_pre_merge.merge(
    responce_df, left_on="Date", right_on="index", how="left", indicator=True
)

In [30]:
X_feautures_final.tail()

,Date,videoTitleSentimentScore_pos,videoTitleSentimentScore_neg,captionSentimentScore_pos,captionSentimentScore_neg,num_days_since_pub,viewcounts_weighted,like_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted,index,percent_change,action,_merge
113,2021-03-15,0.037338,0.033960,0.087764,0.030724,109.768889,15241.801392,1049.344298,0.955442,0.955442,14861.456219,2021-03-15,-0.021599,hold,both
114,2021-03-16,0.037245,0.064280,0.105100,0.034100,108.970000,14870.723784,1032.050839,0.954523,0.954523,14469.862263,2021-03-16,-0.043737,sell,both
115,2021-03-17,0.038080,0.066406,0.102891,0.033063,111.137143,14748.626569,939.470945,0.956001,0.956001,14378.400682,2021-03-17,-0.006551,hold,both
116,2021-03-18,0.051550,0.061970,0.099855,0.031660,107.740000,15558.251680,1049.395704,0.956913,0.956913,15162.712790,NaN,NaN,NaN,left_only
117,2021-03-19,0.042326,0.062851,0.109966,0.036794,110.005714,15417.660309,1121.427479,0.957292,0.957292,15049.514412,NaN,NaN,NaN,left_only


In [32]:
X_feautures_final.to_csv(
    "../00_data/Feature_Built_Sent_Data_22MAR.gz", compression="gzip"
)

In [60]:
nvidia_df.head()

,Unnamed: 0,Index,videoID,datePub,searchedDate,VideoTitle,channelTitle,viewCount,likeCount,dislikeCount,...,captionSentimentScore_com,videoTitleSentimentScore_com,num_days_since_pub,viewcounts_weighted,like_count_weighted,dislike_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted,Date
0,0,0,y4fb_R5Ogw0,2020-10-01 05:16:02+00:00,2020-10-02 01:45:21.553469+00:00,"2x NVIDIA RTX 3090 SLI Benchmarks: 500FPS, 700...",Gamers Nexus,367472,15701,260,...,0.9997,0.2023,1.0,367472.000000,15701.000000,260.000000,0.983710,0.983710,361485.989098,2020-10-02
1,1,1,ALEXVtnNEwA,2020-09-01 17:15:57+00:00,2020-10-02 01:45:22.351801+00:00,NVIDIA GeForce Special Event Livestream,IGN,543115,16860,411,...,1.0000,0.4019,31.0,17519.838710,543.870968,13.258065,0.976203,0.976203,17102.917066,2020-10-02
2,2,2,E98hC9e__Xs,2020-09-01 16:42:07+00:00,2020-10-02 01:45:23.252012+00:00,NVIDIA GeForce RTX 30 Series | Official Launch...,NVIDIA GeForce,1484075,54741,1135,...,1.0000,0.0000,31.0,47873.387097,1765.838710,36.612903,0.979687,0.979687,46900.942857,2020-10-02
3,3,3,nMns8t4OhI0,2020-09-30 23:00:06+00:00,2020-10-02 01:45:23.922084+00:00,NVIDIA Responds to RTX 3080 Crashes...,JayzTwoCents,561238,27766,885,...,0.0000,0.0000,2.0,280619.000000,13883.000000,442.500000,0.969111,0.969111,271950.966947,2020-10-02
4,4,4,AG_ZHi3tuyk,2020-09-16 12:59:59+00:00,2020-10-02 01:45:24.592244+00:00,"Nvidia, you PROMISED! - RTX 3080 Review",Linus Tech Tips,3391565,141852,3961,...,0.9997,0.5461,16.0,211972.812500,8865.750000,247.562500,0.972835,0.972835,206214.585797,2020-10-02


### APPENDIX

In [ ]:
pd.DataFrame(
    columns=[
        "videoTitleSentimentScore_pos",
        "captionSentimentScore_pos",
        "viewcounts_weighted",
        "like_count_weighted",
        "dislike_count_weighted",
        "like_dislike_ratio",
        "like_dislike_ratio_by_viewcounts_weighted",
    ]
)

nvidia_df["videoTitleSentimentScore_pos"] = [
    sid.polarity_scores(x)["pos"] for x in nvidia_df["VideoTitle"]
]

nvidia_df["videoTitleSentimentScore_neg"] = [
    sid.polarity_scores(x)["neg"] for x in nvidia_df["VideoTitle"]
]

nvidia_df["captionSentimentScore_pos"] = [
    sid.polarity_scores(x)["pos"] for x in nvidia_df["captionString"]
]

nvidia_df["captionSentimentScore_neg"] = [
    sid.polarity_scores(x)["neg"] for x in nvidia_df["captionString"]
]

In [ ]:
X_features = pd.DataFrame(
    columns=[
        "videoTitleSentimentScore_pos",
        "captionSentimentScore_pos",
        "viewcounts_weighted",
        "like_count_weighted",
        "dislike_count_weighted",
        "like_dislike_ratio",
        "like_dislike_ratio_by_viewcounts_weighted",
    ]
)

In [ ]:
nvidia_df["num_days_since_pub"] = np.ceil(
    (nvidia_df["searchedDate"] - nvidia_df["datePub"]) / np.timedelta64(1, "D")
)

In [ ]:
nvidia_df["viewcounts_weighted"] = (
    nvidia_df["viewCount"] / nvidia_df["num_days_since_pub"]
)

In [ ]:
nvidia_df["like_count_weighted"] = (
    nvidia_df["likeCount"] / nvidia_df["num_days_since_pub"]
)

In [ ]:
nvidia_df["like_count_weighted"] = (
    nvidia_df["dislikeCount"] / nvidia_df["num_days_since_pub"]
)